In [1]:
!pip install transformers

     |████████████████████████████████| 778kB 8.7MB/s 
     |████████████████████████████████| 1.1MB 25.6MB/s 
     |████████████████████████████████| 890kB 28.5MB/s 
     |████████████████████████████████| 3.0MB 52.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=8d2ea722e319ee5ff5799755e8f33900ad665bbb154ffcc7440f23b070f12a03
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())

Version:  2.2.0
Eager mode:  True


In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [4]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBC5ZD1/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBC5ZD1/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBC5ZD1/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))
train_examples[:2]

Training entries: 25000, test entries: 25000


array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot 

In [6]:
train_labels[:2]

array([0, 0])

In [7]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard


def load_data(examples, targets, tokenizer):

    reviews, labels = [], []
    input_ids, token_type_ids, attention_masks = [], [], []
    for example, label in zip(examples, targets):
      reviews.append(str(example).strip())
      labels.append(label)

    X = tokenizer(reviews, 
                       padding=True, 
                       truncation=True, 
                       return_tensors="tf")

    y = np.array(labels)
    
    print ("begin loding data...")
    data = {}
    data["X"] = X
    data["input_ids"] = X["input_ids"]
    data["token_type_ids"] = X["token_type_ids"]
    data["attention_mask"] = X["attention_mask"]
    data["y"] = y
    data["tokenizer"] = tokenizer
    data["int2label"] =  {0: "negative", 1: "positive"}
    data["label2int"] = {"negative": 0, "positive": 1}

    return data

In [8]:
from transformers import BertTokenizer, TFBertForSequenceClassification

tokenizer =  BertTokenizer.from_pretrained('bert-base-uncased')
bert = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
train_data = load_data(train_examples[:5], train_labels, tokenizer)

begin loding data...


In [10]:
out = bert(train_data["X"])
out

(<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
 array([[ 0.05834597, -0.19027337],
        [-0.04664063, -0.22685416],
        [ 0.00268216, -0.22869053],
        [-0.08591065, -0.23248336],
        [ 0.06754325, -0.22161664]], dtype=float32)>,)

In [11]:
tf_predictions = tf.nn.softmax(out[0], axis=-1)
tf_predictions

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.56183666, 0.43816337],
       [0.5449318 , 0.45506817],
       [0.5575865 , 0.44241348],
       [0.5365777 , 0.4634223 ],
       [0.57179046, 0.42820954]], dtype=float32)>

In [12]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

train_data = load_data(train_examples[:100], train_labels, tokenizer)
train_dataset = tf.data.Dataset.from_tensor_slices((train_data["input_ids"], train_data["token_type_ids"], train_data["attention_mask"], train_data["y"])).map(map_example_to_dict).shuffle(32).batch(12)
test_data = load_data(test_examples[:100], test_labels, tokenizer)
test_dataset = tf.data.Dataset.from_tensor_slices((test_data["input_ids"], test_data["token_type_ids"], test_data["attention_mask"], test_data["y"])).map(map_example_to_dict).shuffle(32).batch(12)
train_dataset

begin loding data...
begin loding data...


<BatchDataset shapes: ({input_ids: (None, 512), token_type_ids: (None, 512), attention_mask: (None, 512)}, (None,)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>

In [13]:
learning_rate = 2e-5
number_of_epochs = 5
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
bert.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=[metric])

In [37]:
bert_history = bert.fit(train_dataset, 
                        epochs=number_of_epochs, 
                        validation_data=test_dataset)

!mkdir -p saved_model
bert.save_pretrained('saved_model/my_model')

bert_history

Epoch 1/5
9/9 [==============================] - 14s 2s/step - loss: 0.0621 - accuracy: 0.9800 - val_loss: 0.5461 - val_accuracy: 0.8000
Epoch 2/5
9/9 [==============================] - 15s 2s/step - loss: 0.0233 - accuracy: 1.0000 - val_loss: 0.8498 - val_accuracy: 0.6700
Epoch 3/5
9/9 [==============================] - 15s 2s/step - loss: 0.0177 - accuracy: 1.0000 - val_loss: 0.6361 - val_accuracy: 0.7800
Epoch 4/5
9/9 [==============================] - 15s 2s/step - loss: 0.0148 - accuracy: 1.0000 - val_loss: 0.6535 - val_accuracy: 0.7700
Epoch 5/5
9/9 [==============================] - 15s 2s/step - loss: 0.0112 - accuracy: 1.0000 - val_loss: 0.7122 - val_accuracy: 0.7800


In [38]:
text = "The movie is awesome!"
text = train_data["tokenizer"]([text], padding=True, truncation=True, return_tensors="tf")
prediction = bert(text)

print("Prediction:",  tf.nn.softmax(prediction[0], axis=-1))
print(train_data["int2label"][np.argmax(prediction)])

Prediction: tf.Tensor([[0.27672294 0.7232771 ]], shape=(1, 2), dtype=float32)
positive


In [39]:
!ls saved_model/my_model

assets	config.json  saved_model.pb  tf_model.h5  variables


In [44]:
new_model =  TFBertForSequenceClassification.from_pretrained('saved_model/my_model')
new_model.summary()

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_151 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [45]:
text = "The movie is awesome!"
text = train_data["tokenizer"]([text], padding=True, truncation=True, return_tensors="tf")
prediction = new_model(text)

print("Prediction:",  tf.nn.softmax(prediction[0], axis=-1))
print(train_data["int2label"][np.argmax(prediction)])

Prediction: tf.Tensor([[0.27672294 0.7232771 ]], shape=(1, 2), dtype=float32)
positive


In [49]:
learning_rate = 2e-5
number_of_epochs = 5
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
new_model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=[metric])
result = new_model.evaluate(test_dataset)
loss, prediction_scores = result
print('loss is: {}'.format(loss))
print('prediction_scores is: {}'.format(prediction_scores))

9/9 [==============================] - 3s 376ms/step - loss: 0.7122 - accuracy: 0.7800
loss is: 0.7121712565422058
prediction_scores is: 0.7799999713897705
